In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import spacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

Obtention des données

In [6]:
import Outils.data_extraction.mongo_db as mg
tweet = mg.connection().tweet

Connexion réussie <bound method MongoClient.database_names of MongoClient(host=['application-shard-00-02.wl73u.mongodb.net:27017', 'application-shard-00-00.wl73u.mongodb.net:27017', 'application-shard-00-01.wl73u.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-n3r5s8-shard-0', ssl=True)>


In [5]:
def create_set(tweet,nom):
    df=[]
    for i in tweet.tweets.find({"critic_word":nom}):
        row=[]
        row.append(mg.bson_to_json_file(i)['user_id'])
        row.append(mg.bson_to_json_file(i)['id'])
        row.append(mg.bson_to_json_file(i)['text'])
        df.append(row)
    df=pd.DataFrame(df,columns=["id_user","id","text"])
    return df.drop_duplicates(subset="text",ignore_index=True)


In [19]:
nom = tweet.tweets.find({"critic_word":'feel alone depressed'})
mg.bson_to_json_file(name)

'feel alone depressed'

In [7]:
name='feel alone depressed'
data=create_set(tweet,name)

In [39]:
data.to_pickle('synthese_emotion/syntese '+name+'.pkl')

In [22]:
def name_to_pickle(name,tweet):
    data=create_set(tweet,name)
    data.to_pickle('synthese_emotion/syntese '+name+'.pkl')

In [13]:
list_search_words = ["feel alone depressed","i feel helpless", "i feel sad","i feel empty","sleeping a lot lately","i feel irritable",
    "depressed alcohol","sertraline","Zoloft","Prozac","pills depressed","suicide once more", "pain suicide","mom suicide tried","friend suicide","sister suicide tried","Brother suicide tried",
    "suicide attempted sister", "thought suicide before", "had thoughts suicide","had thoughts killing myself", "i want to commit suicide", "stop cutting myself",
    "i'm being bullied", "feel bullied i'm", "stop bullying me","always getting bullied", "gun suicide", "been diagnosed anorexia", "i diagnosed OCD", "I diagnosed bipolar",
    "dad fight again", "parents fight again", "i impulsive", "i'm impulsive"]

In [45]:
for i in list_search_words:
    name_to_pickle(i, tweet)

In [20]:
pd.read_pickle('synthese_emotion\syntese depressed alcohol.pkl')

,id_user,id,text
0,1019298542,1206348731341312002,For me... when depressed food is what you call...
1,1363224626,1206348398753910787,RT @emo_daydream: Am i depressed on Sundays fr...
2,335429185,1206346296250052610,Am i depressed on Sundays from the alcohol or ...
3,1528609417,1206336958907203591,The hard part about a broken heart is that no ...
4,976210406304813056,1206297878509031425,RT @Suhail: The portrayal of alcohol in entert...
...,...,...,...
259,15318541,1238133060941615107,If you’re depressed and anxious about our fuck...
260,716136265,1238129165972516865,We Are a depressed generation hiding behind Al...
261,909272440739528706,1238128026291965952,@MarkoftheWest @Sigebrand How ironic is it tha...
262,182756674,1238102203937837056,@AlexWhite1812 @RobinWigg My advice would be t...


Analyse

In [22]:
filename='critic_word_feel_alone_depressed.pkl'

In [104]:
df = pd.read_csv('emotion.csv', sep=';')
tweet_set=pd.read_pickle(filename)

In [89]:
def emotion_phrase(phrase):
    liste_emotion = []
    phrase = nlp(phrase)
    for token in phrase:
        mot = token.text
        for i in range(len(df)):
            if(df.loc[i, 'English (en)'] == mot):
                liste_emotion.append(df.loc[i,:].tolist())
    return pd.DataFrame(liste_emotion, columns=['English (en)', 'Positive' ,'Negative','Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust'])

def boucle_ensemble_tweet(df):
    result_df= pd.DataFrame(columns=['English (en)', 'Positive' ,'Negative','Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust','id'])
    #emotion_df=pd.DataFrame(columns=['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust','id'])
    #positivite_df=pd.DataFrame(columns=['Positive' ,'Negative','id'])
    for i in range(len(df)):
        text=df.iloc[i,2]
        resultat= emotion_phrase(text)
        resultat = resultat.sum()
        resultat = pd.DataFrame(resultat).transpose()
        resultat['id']=df.iloc[i,1]
        result_df=result_df.append(resultat,ignore_index=True)

    return result_df

#Probabilité
def probabilisation_emotion(df):
    emotion_df = pd.DataFrame(columns=['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust','id'])
    for i in range(len(df)):
        line = df[['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust']].iloc[i]
        try :
            line = line / line.sum()
        except ZeroDivisionError:
            print('Division')
        line['id']=df[['id']].iloc[i]
        emotion_df = emotion_df.append(line,ignore_index=True)
    return emotion_df

In [91]:
tweet_set.iloc[1,2]

'Job seeking can be incredibly stressful.\n\nIt can make you feel lost.\n\nHurt.\n\nAlone.\n\nNeed I say, depressed.\n\nMany o… https://t.co/XOVbx4fJFQ'

In [123]:
df = emotion_phrase(tweet_set.iloc[10,2])

In [119]:
df

,English (en),Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,feel,0,0,0,0,0,0,0,0,0,0
1,depressed,0,1,1,0,0,1,0,1,0,0


In [65]:
df=df[['English (en)','Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust']]

In [120]:
new_df=pd.melt(df,id_vars="English (en)",value_vars=list(df.columns[1:]),var_name="Emotion")
new_df

,English (en),Emotion,value
0,feel,Positive,0
1,depressed,Positive,0
2,feel,Negative,0
3,depressed,Negative,1
4,feel,Anger,0
5,depressed,Anger,1
6,feel,Anticipation,0
7,depressed,Anticipation,0
8,feel,Disgust,0
9,depressed,Disgust,0


In [121]:
without_0=new_df[new_df["value"]!=0].iloc[:,:-1]
without_0

,English (en),Emotion
3,depressed,Negative
5,depressed,Anger
11,depressed,Fear
15,depressed,Sadness


In [ ]:
df[['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust']]
new_df=df[['English (en)','Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust']]
import plotly_express as px
import plotly.io as plt

In [46]:
def emotion_phrase(phrase):
    liste_emotion = []
    phrase = nlp(phrase)
    for token in phrase:
        mot = token.text
        for i in range(len(df)):
            if(df.loc[i, 'English (en)'] == mot):
                liste_emotion.append(df.loc[i,:].tolist())
    return pd.DataFrame(liste_emotion, columns=['English (en)', 'Positive' ,'Negative','Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust'])





In [122]:
fig = px.parallel_categories(without_0)
fig.show()
fig.write_html('test.html')

In [45]:
variable_pour_Romain=plt.to_html(fig)

In [7]:
data = boucle_ensemble_tweet(tweet_set)
data = probabilisation_emotion(data)

KeyboardInterrupt: 

In [17]:
data.to_pickle("emotion_analysis_"+filename)

NameError: name 'data' is not defined

In [6]:
list_search_words = ["stop cutting myself",
    "i'm being bullied", "feel bullied i'm", "stop bullying me","always getting bullied", "gun suicide", "been diagnosed anorexia", "i diagnosed OCD", "I diagnosed bipolar",
    "dad fight again", "parents fight again", "i impulsive", "i'm impulsive"]

In [7]:
def emotion_extraction(word):
    print(word)
    base_url= "synthese_emotion/syntese "
    data=pd.read_pickle(base_url+word+'.pkl')
    first_step = boucle_ensemble_tweet(data)
    second_step = probabilisation_emotion(first_step)
    second_step.to_pickle("emotion_extraction/emotion_analysis_"+word+'.pkl')
    second_step.to_csv("emotion_extraction/emotion_analysis_"+word+'.csv')
    

In [15]:
for i in list_search_words:
    emotion_extraction(i)

Zoloft
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Prozac
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Division
Divis

In [18]:
pd.read_pickle("emotion_extraction/emotion_analysis_i feel sad.pkl")

,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"id 1206363247323992064 Name: 0, dtype: object"
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"id 1206362722667040769 Name: 1, dtype: object"
2,0.074074,0.148148,0.074074,0.074074,0.222222,0.111111,0.111111,0.185185,"id 1206362664634593281 Name: 2, dtype: object"
3,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,"id 1206362627280166914 Name: 3, dtype: object"
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"id 1206362537526255616 Name: 4, dtype: object"
...,...,...,...,...,...,...,...,...,...
328,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,"id 1239335150816821248 Name: 328, dtype: ob..."
329,0.250000,0.000000,0.250000,0.250000,0.000000,0.250000,0.000000,0.000000,"id 1239335074698366976 Name: 329, dtype: ob..."
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"id 1239334875309670403 Name: 330, dtype: ob..."
331,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,"id 1239334758070411264 Name: 331, dtype: ob..."
